<a href="https://colab.research.google.com/github/CarlosV1204/CarlosV1204/blob/main/ProyectoNoteSeq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#TensorFlow, Note-Seq, Librosa y Plotting
!pip install tensorflow note-seq librosa matplotlib

!pip install MIDIUtil streamlit # Streamlit requiere un servidor aparte

In [ ]:
# CELDAS DE INSTALACIÓN ADICIONAL
print("Verificando herramientas de síntesis de audio...")
# fluidsynth: El motor de síntesis de audio nativo (debe estar instalado)
!apt-get install fluidsynth -qq > /dev/null

# Se omite soundfont_synth, ya que no existe.

# Descargar un SoundFont de dominio público para la síntesis (Esencial)
SF2_PATH = 'FluidR3_GM.sf2'
# Descargar el SoundFont, ya que Colab puede no tener uno predeterminado con ese nombre
!wget -q -O {SF2_PATH} 'http://www.synthfont.com/FluidR3_GM.sf2'

print("✅ Herramientas de síntesis configuradas.")

Verificando herramientas de síntesis de audio...
✅ Herramientas de síntesis configuradas.


In [ ]:
# CELDAS DE VERIFICACIÓN
try:
    import tensorflow as tf
    import note_seq
    import librosa
    import midiutil
    import streamlit
    import matplotlib
    import numpy as np

    # Muestra la versión de TensorFlow
    print("✅ TensorFlow versión:", tf.__version__)

    # Verifica que Note-Seq se puede inicializar
    _ = note_seq.NoteSequence()
    print("✅ Note-Seq está correctamente instalado.")

    print("\n¡El entorno está listo para tu proyecto de Composición Musical!")

except ImportError as e:
    print(f"❌ ERROR: Falló la importación de una librería. Detalle: {e}")

✅ TensorFlow versión: 2.19.0
✅ Note-Seq está correctamente instalado.

¡El entorno está listo para tu proyecto de Composición Musical!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Ruta del archivo de audio base en tu Drive
INPUT_AUDIO_FILE = '/content/drive/MyDrive/ProyectoPI/audios/ARMONICA.mp3'

# Comprobar si el archivo existe
import os
if os.path.exists(INPUT_AUDIO_FILE):
    print(f"✅ Archivo de entrada encontrado en Drive: {INPUT_AUDIO_FILE}")
else:
    print(f"❌ ERROR: Archivo no encontrado. Verifica la ruta: {INPUT_AUDIO_FILE}")

✅ Archivo de entrada encontrado en Drive: /content/drive/MyDrive/ProyectoPI/audios/ARMONICA.mp3


In [ ]:
import librosa

# Parámetros de optimización
TARGET_SR = 16000     # Usar una tasa de muestreo más baja (16kHz es suficiente para pitch), cuántas muestras de audio se toman por segundo. El archivo los tomaba de 48Khz
HOP_LENGTH = 512 * 2  # Aumenta el tamaño del salto (reduce el número de cálculos de pitch)

if INPUT_AUDIO_FILE:
    print(f"Iniciando procesamiento de audio (SR={TARGET_SR} Hz)...")

    # 1. Cargar la forma de onda con remuestreo
    y, sr = librosa.load(INPUT_AUDIO_FILE, sr=TARGET_SR) #y es el array de la forma de onda y sr es la tasa de muestreo real 16Khz

    # 2. Estimar la frecuencia fundamental (f0) con el hop_length optimizado
    print("Estimando frecuencia fundamental (PYIN)...")
    f0, voiced_flag, voiced_probabilities = librosa.pyin(
        y,
        fmin=librosa.note_to_hz('C2'), #Las notas empienzas de do2 a un do7
        fmax=librosa.note_to_hz('C7'),
        sr=sr,
        hop_length=HOP_LENGTH
    )

    # 3. Convertir las frecuencias (Hz) a notas MIDI
    midi_notes = librosa.hz_to_midi(f0) #y al final todas estas notas se pasan a formato midi

    print(f"✅ Extracción de notas MIDI inicial completada. Duración: {librosa.get_duration(y=y, sr=sr):.2f} segundos.")
else:
    print("No se puede continuar sin un archivo de entrada válido.")

Iniciando procesamiento de audio (SR=16000 Hz)...
Estimando frecuencia fundamental (PYIN)...
✅ Extracción de notas MIDI inicial completada. Duración: 91.15 segundos.


In [ ]:
import note_seq
from note_seq.constants import DEFAULT_QUARTERS_PER_MINUTE

# --- Variables de Respaldo ---c
DEFAULT_BPM = 120.0

# 1. Detección de Tempo (Corregido)
try: # ES el manejor de errores con librosa
    onset_env = librosa.onset.onset_detect(y=y, sr=sr, units='time')
    tempo_array, _ = librosa.feature.rhythm.tempo(y=y, sr=sr, onset_envelope=librosa.onset.onset_strength(y=y, sr=sr))
    detected_tempo = tempo_array[0]
    print(f"✅ Tempo detectado: {detected_tempo:.2f} BPM")

except Exception as e:
    detected_tempo = DEFAULT_BPM
    print(f"⚠️ Detección de Tempo fallida. Usando tempo por defecto: {DEFAULT_BPM:.2f} BPM (Error: {e})")

# --- 2. Inicializar la NoteSequence ---
times = librosa.times_like(f0, sr=sr, hop_length=HOP_LENGTH) #calcular tiempo, secuencia y duracion
sequence = note_seq.NoteSequence()
sequence.total_time = librosa.get_duration(y=y, sr=sr)

# LÍNEAS CORREGIDAS: Asignación de tempo
sequence.tempos.add().qpm = detected_tempo
sequence.tempos[0].time = 0.0

# Lógica de Agrupación de Notas (Consolidación) ---
notes_list = []
current_start_time = 0.0
current_pitch = -1

#Recorre cada pequeña porción de
#tiempo (frame) donde se midió el pitch tono (midi_notes)
for i, pitch in enumerate(midi_notes):
    rounded_pitch = int(np.round(pitch)) if voiced_flag[i] else -1
    current_time_step = times[i]
#Convierte el pitch a un valor MIDI entero (60, 61, etc.) si se detectó
      #una voz (voiced_flag). Si es silencio o ruido, se asigna -1
    if rounded_pitch == current_pitch:
        continue

    # Finaliza la nota anterior
    if current_pitch != -1 and current_start_time < current_time_step:

        notes_list.append(sequence.notes.add(
            pitch=current_pitch,
            start_time=current_start_time,
            end_time=current_time_step,
            velocity=100, # AUMENTO DE VELOCIDAD DE LA CANCIÓN DE ENTRADA (80 -> 100)
            instrument=25 # Guitarra Acústica/Voz Base
        ))

    # Inicia la nueva nota o silencio
    current_pitch = rounded_pitch
    current_start_time = current_time_step

# Asegura que la última nota se cierre
if current_pitch != -1 and current_start_time < sequence.total_time:
    sequence.notes.add(
        pitch=current_pitch,
        start_time=current_start_time,
        end_time=sequence.total_time,
        velocity=100, # AUMENTO DE VELOCIDAD DE LA CANCIÓN DE ENTRADA (80 -> 100)
        instrument=25
    )

print("✅ NoteSequence de Melodía Base creada con VELOCIDAD AUMENTADA.")

ModuleNotFoundError: No module named 'note_seq'

In [ ]:
# CELDAS DE ANÁLISIS (MÓDULO 6.1)

# El código asume que las variables y, sr, sequence, detected_tempo están definidas
# y que ya se ejecutó la construcción de la NoteSequence (Celda 3).

import librosa
import numpy as np
from collections import Counter

print("\n--- INICIANDO ANÁLISIS MUSICAL ---")


#Convierte la señal de audio (y) en una representación
#que muestra la intensidad de las 12 clases de pitch (C, C#, D, etc.)
#1. Análisis de Tonalidad (Key)
print("Iniciando análisis armónico (Cromograma)...")
chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
chroma_mean = np.mean(chroma, axis=1)


#Encuentra la clase de pitch
#que tiene la mayor energía promedio en toda la canción.
# Determinar la tonalidad dominante
key_index = np.argmax(chroma_mean)
#Convierte el índice numérico (0-11) en el nombre de la nota (ej. 'A', 'C#').
TONALIDAD = librosa.midi_to_note(key_index + 60, octave=False)
print(f"✅ Tonalidad detectada (Pitch Class): {TONALIDAD}")



#Calcula la desviación estándar (Std Dev) de la energía entre las 12 clases de pitch.
#Un valor: Bajo ($\approx 0$): Indica que
#solo unas pocas notas son dominantes (acordes simples, claros). Alto:
#Indica que muchas notas están presentes (acordes complejos o disonantes).
# 2. Análisis de Complejidad Armónica (Informará al Piano/Cuerdas) ---
COMPLEJIDAD_ARMONICA = np.std(chroma_mean)
print(f"✅ Complejidad Armónica (Std Dev): {COMPLEJIDAD_ARMONICA:.3f}")


#3. Análisis de Densidad Rítmica (Informará a la Percusión) ---
DURACION_SEGUNDOS = sequence.total_time
NUM_NOTAS = len(sequence.notes)
DENSIDAD_RITMICA = NUM_NOTAS / DURACION_SEGUNDOS if DURACION_SEGUNDOS > 0 else 0
print(f"✅ Densidad Rítmica (Notas/seg): {DENSIDAD_RITMICA:.2f}")

# Guardar la Tonalidad MIDI (necesaria para el Bajo/Armonía)

#Convierte la TONALIDAD detectada a un número MIDI
#específico (en la octava 3, ej: C3 = 48)
global KEY_PITCH
KEY_PITCH = librosa.note_to_midi(TONALIDAD + '3')

# Inicializar la duración del compás (necesaria para las siguientes celdas)
BAR_DURATION = 60.0 / detected_tempo * 4
print(f"Tempo base: {detected_tempo:.2f} BPM. Duración del compás: {BAR_DURATION:.2f} s.")

# Clonar la secuencia base para la maqueta final
maqueta_final = sequence


--- INICIANDO ANÁLISIS MUSICAL ---
Iniciando análisis armónico (Cromograma)...
✅ Tonalidad detectada (Pitch Class): G
✅ Complejidad Armónica (Std Dev): 0.111
✅ Densidad Rítmica (Notas/seg): 4.56
Tempo base: 120.00 BPM. Duración del compás: 2.00 s.


In [ ]:
# MÓDULO 6.2: Pista de Percusión

# Reutilizar variables globales
#Asegura que el código pueda leer el ritmo (BAR_DURATION, DENSIDAD_RITMICA)
#y escribir las nuevas notas directamente en la variable maqueta_final
global maqueta_final, BAR_DURATION, DENSIDAD_RITMICA

# Instrumentos: Bombo (MIDI: 36 Kick Drum), Caja (MIDI: 38 Snare Drum), Canal de percusión (10)
KICK_PITCH = 36
SNARE_PITCH = 38
current_time = 0.0


#AGREGAR CONSTANTES PARA ELECCION DEL USUARIO, O BARRA PARA ESCOGER LA VELOCIDAD

#IGUAL DURACION DEL BEAT

# Ajustar la intensidad de la percusión (Módulo de IA/Lógica)
# Si la densidad es alta (>4 notas/s), la batería será más fuerte y "ocupada"
VELOCIDAD_PERCUSION = 100 if DENSIDAD_RITMICA > 4 else 80 # la velocidad es mas si rebasa de los 4 beats
DURACION_BEAT = BAR_DURATION / 4  #Divide el total del compas entre 4 beats

print("Generando Pista de Percusión...")
while current_time < maqueta_final.total_time:
    # BOMBO (Beats 1 y 3)Toca el golpe de bajo en los downbeats
    maqueta_final.notes.add(pitch=KICK_PITCH, start_time=current_time, end_time=current_time + 0.01, velocity=VELOCIDAD_PERCUSION, instrument=10, is_drum=True)
    maqueta_final.notes.add(pitch=KICK_PITCH, start_time=current_time + DURACION_BEAT * 2, end_time=current_time + DURACION_BEAT * 2 + 0.01, velocity=VELOCIDAD_PERCUSION, instrument=10, is_drum=True)

    # CAJA (Beats 2 y 4) "boom-chak-boom-chak"
    maqueta_final.notes.add(pitch=SNARE_PITCH, start_time=current_time + DURACION_BEAT, end_time=current_time + DURACION_BEAT + 0.01, velocity=VELOCIDAD_PERCUSION - 10, instrument=10, is_drum=True)
    maqueta_final.notes.add(pitch=SNARE_PITCH, start_time=current_time + DURACION_BEAT * 3, end_time=current_time + DURACION_BEAT * 3 + 0.01, velocity=VELOCIDAD_PERCUSION - 10, instrument=10, is_drum=True)

    current_time += BAR_DURATION

print("✅ Pista de Percusión generada y añadida a la maqueta final.")

Generando Pista de Percusión...
✅ Pista de Percusión generada y añadida a la maqueta final.


In [ ]:
# MÓDULO 6.2: Pista de Cuerdas (Violines/Pads)

# Reutilizar variables globales
global maqueta_final, BAR_DURATION, KEY_PITCH

# Instrumento: Cuerdas (Violines, MIDI: 49)
STRINGS_INSTRUMENT = 49
current_time = 0.0

# Definir la nota base para el pad de cuerdas (la tónica detectada)
STRINGS_PITCH = KEY_PITCH + 12 # Una octava más alta que el bajo

print("Generando Pista de Cuerdas (Violines Pads)...")
while current_time < maqueta_final.total_time:
    # Toca la nota base de la tonalidad durante todo el compás (efecto pad)
    maqueta_final.notes.add(
        pitch=STRINGS_PITCH,
        start_time=current_time,
        end_time=current_time + BAR_DURATION,
        velocity=70, # Suave para un pad
        instrument=STRINGS_INSTRUMENT
    )
    current_time += BAR_DURATION

print("✅ Pista de Cuerdas (Violines) generada y añadida a la maqueta final.")

Generando Pista de Cuerdas (Violines Pads)...
✅ Pista de Cuerdas (Violines) generada y añadida a la maqueta final.


In [ ]:
# MÓDULO 6.2: Pista de Piano / Bajo Armónico

# Reutilizar variables globales
global maqueta_final, BAR_DURATION, KEY_PITCH, COMPLEJIDAD_ARMONICA

# Instrumento: Piano Eléctrico (MIDI: 5)
PIANO_INSTRUMENT = 5
current_time = 0.0

# Definir la nota base (el bajo que toca la fundamental)
PIANO_PITCH = KEY_PITCH - 12 # Una octava más baja (Bassline)

print("Generando Pista de Bajo Armónico (Piano/Bassline)...")

# Lógica de Generación de Bajo/Armonía:
if COMPLEJIDAD_ARMONICA < 0.2:
    # Si la entrada es simple (bajo), crea un bajo simple con el piano.
    PATRON_BAJO = [0.0, BAR_DURATION / 2.0] # Beat 1 y Beat 3
    VELOCIDAD = 85
else:
    # Si la entrada es compleja, usa el piano para tocar acordes simples
    PATRON_BAJO = [0.0] # Toca solo en el Beat 1
    VELOCIDAD = 75


while current_time < maqueta_final.total_time:
    for offset in PATRON_BAJO:
        if current_time + offset < maqueta_final.total_time:
            # 1. Añadir la nota fundamental (Bajo/Tónica)
            maqueta_final.notes.add(
                pitch=PIANO_PITCH,
                start_time=current_time + offset,
                end_time=current_time + offset + BAR_DURATION * 0.15,
                velocity=VELOCIDAD,
                instrument=PIANO_INSTRUMENT
            )

            # 2. Si la complejidad es alta, se puede añadir una nota de acorde (ej. tercera mayor)
            if COMPLEJIDAD_ARMONICA >= 0.2 and offset == 0.0:
                 maqueta_final.notes.add(
                    pitch=PIANO_PITCH + 4, # Tercera mayor
                    start_time=current_time + offset,
                    end_time=current_time + offset + BAR_DURATION * 0.5,
                    velocity=VELOCIDAD - 10,
                    instrument=PIANO_INSTRUMENT
                )


    current_time += BAR_DURATION

print("✅ Pista de Piano/Bajo Armónico generada y añadida a la maqueta final.")

Generando Pista de Bajo Armónico (Piano/Bassline)...
✅ Pista de Piano/Bajo Armónico generada y añadida a la maqueta final.


In [ ]:
# MÓDULO DE EXPORTACIÓN FINAL (6.4)

from google.colab import files
import note_seq

# NOTA IMPORTANTE: La variable 'maqueta_final' ya contiene:
# 1. Pista 1: Melodía Base (Guitarra/Voz) [Instrumento 25]
# 2. Pista 2: Percusión (Batería) [Instrumento 10, is_drum=True]
# 3. Pista 3: Cuerdas (Violines Pad) [Instrumento 49]
# 4. Pista 4: Piano/Bajo Armónico [Instrumento 5]

OUTPUT_FILE = "maqueta_completa_final.mid"

# Exportar la NoteSequence a un archivo MIDI
# Dado que la NoteSequence es un contenedor de todas las notas,
# el archivo MIDI resultante incluye todas las pistas añadidas.
note_seq.sequence_proto_to_midi_file(maqueta_final, OUTPUT_FILE)
print(f"\n✅ Proceso completado. La maqueta completa ha sido exportada a: {OUTPUT_FILE}")
print("El archivo MIDI contiene la melodía original + las 3 pistas de acompañamiento.")

# Descargar el archivo
files.download(OUTPUT_FILE)


✅ Proceso completado. La maqueta completa ha sido exportada a: maqueta_completa_final.mp3
El archivo MIDI contiene la melodía original + las 3 pistas de acompañamiento.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import note_seq
from pydub import AudioSegment
import os

# Archivos de trabajo
MIDI_TEMP_FILE = "maqueta_temp.mid"
OUTPUT_WAV_FILE = "maqueta_completa.wav"
OUTPUT_MP3_FILE = "maqueta_completa.mp3"
SF2_PATH = 'FluidR3_GM.sf2'

# --- 1. Exportar la NoteSequence a un archivo MIDI temporal ---
print("1. Exportando NoteSequence a MIDI temporal...")
note_seq.sequence_proto_to_midi_file(maqueta_final, MIDI_TEMP_FILE)

# --- 2. Síntesis de Audio (MIDI -> WAV) usando fluidsynth ---
print("2. Iniciando síntesis de audio (MIDI -> WAV con FluidSynth)...")
# El comando fluidsynth usa el SoundFont para renderizar el archivo MIDI
!fluidsynth -ni {SF2_PATH} {MIDI_TEMP_FILE} -F {OUTPUT_WAV_FILE} -r 44100 -g 1.0 > /dev/null

# --- 3. Conversión a MP3 y Descarga ---
print("3. Convirtiendo WAV a MP3 y descargando...")

# Cargar el archivo WAV con pydub
audio_segment = AudioSegment.from_wav(OUTPUT_WAV_FILE)

# Exportar como MP3
audio_segment.export(OUTPUT_MP3_FILE, format="mp3", bitrate="192k")

print(f"✅ Archivo final MP3 exportado a: {OUTPUT_MP3_FILE}")

# --- 4. Descarga y Limpieza ---
from google.colab import files
files.download(OUTPUT_MP3_FILE)

# Limpieza de archivos temporales
os.remove(MIDI_TEMP_FILE)
os.remove(OUTPUT_WAV_FILE)
print("Archivos temporales eliminados.")

1. Exportando NoteSequence a MIDI temporal...
2. Iniciando síntesis de audio (MIDI -> WAV con FluidSynth)...
fluidsynth: error: fluid_is_soundfont(): expected RIFF chunk id '0x46464952' but got '0x6F64213C'.
Parameter 'FluidR3_GM.sf2' not a SoundFont or MIDI file or error occurred identifying it.
3. Convirtiendo WAV a MP3 y descargando...
✅ Archivo final MP3 exportado a: maqueta_completa.mp3


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Archivos temporales eliminados.
